topic modeling: statistical modeling for discovering the abstract "topics" that occur in a collection of documents

LDA(Latent Dirichlet Allocation): an example of topic model, used to classify text to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

In this project, I will practice applying LDA to a set of documents and split them into topics.

**Agenda**:
* the data
* data pre-processing
* bag of words on the data set
* 

**The Data**

The dataset contains over one millioin news headlines published over 15 years.

In [5]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [11]:
data_text['headline_text'][0]

'aba decides against community broadcasting licence'

In [7]:
len(documents)

1103663

**Data Pre-processing**

steps:
* tokenization: split text into sentences and sentences into words; lowercase words; remove punctuation
* remove words with fewer than 3 char
* remove stopwords
* lemmatize words: words in third person are changed to first person; verbs in past and future tenses are changed into present
* stem words: words are reduced to their root form

In [19]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ruihu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

perform lemmatize and stem preprocessing steps on text

In [21]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

remove stopwords and set threshold for char length

In [16]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

select a document to preview after preprocessing

In [22]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


preprocess the headlines, save results as 'processed_docs'

In [24]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

**Bag of words on the datasets**

create a dictionary containing the number of times a word appears in the training set

In [28]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


'overtim'

filter out tokens that appear in:
* less than 15 documents
* or more than 50% of the total corpus size
* keep the first 100000 most frequent tokens

In [26]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

gensim doc2bow: for each document we create a dictionary reporting how many words and how many times those words appear, save this to 'bow_corpus'

In [62]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]
#word id as key, word freq as value

[(77, 1), (116, 1), (495, 1), (4452, 1)]

preview bag of words for sample processed_docs

In [61]:
bow_doc_4310 = bow_corpus[8]

for i in range(len(bow_doc_4310)):
    print("word {} (\"{}\") appears in {} time".format(bow_doc_4310[i][0],
                                                      dictionary[bow_doc_4310[i][0]],
                                                      bow_doc_4310[i][1]))

word 11 ("aust") appears in 1 time
word 33 ("wast") appears in 1 time
word 34 ("address") appears in 1 time
word 35 ("council") appears in 1 time
word 36 ("iraq") appears in 1 time
